## Build and Push Container

Based on: https://github.com/serengil/deepface/blob/master/deepface/api/src/modules/core/routes.py

In [ ]:
%%sh

image_name=deepface 
bash ./build-and-push.sh $image_name

## Deploy SageMaker Endpoint

In [ ]:
import sagemaker
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

role = sagemaker.get_execution_role()

In [ ]:
image = "807422752601.dkr.ecr.eu-west-1.amazonaws.com/deepface:latest"

In [ ]:
model = sagemaker.model.Model(
    image_uri=image,
    role=role,
    name="deepface"
)

In [ ]:
# Use serverles inference for dev/test
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144, 
    max_concurrency=5,
)

model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    # serverless_inference_config=serverless_config,
    endpoint_name="deepface-endpoint"
    )

## Call Endpoint using boto3

In [ ]:
import base64
import cv2
import boto3
import json

sm_runtime = boto3.client("sagemaker-runtime")

In [ ]:
# Load image in base64
def get_image_in_base64(image_file: str) -> str:
    img = cv2.imread(image_file)
    string = base64.b64encode(cv2.imencode('.jpg', img)[1]).decode()
    img_base65 = f"data:image/jpg;base64,{string}"
    return img_base65

img1_base64: str = get_image_in_base64("steve-jobs.jpg")
img2_base64: str = get_image_in_base64("steve-2.jpg")

In [ ]:
%%time

payload = {
    "service": "represent",
    "img_path": img_base64, 
}

response = sm_runtime.invoke_endpoint(
    EndpointName="face-recognition-dev-serverless",
    Body=bytes(json.dumps(payload), encoding="utf-8"),
    ContentType='application/json',
)

response_json = json.loads(response["Body"].read().decode())

In [ ]:
response_json

In [ ]:
%%time

payload = { # also supports analyze, verify - 
    "service": "analyze",
    "img_path": img_base64, 
    "actions": ["gender"]
}

response = sm_runtime.invoke_endpoint(
    EndpointName="face-recognition-dev-serverless",
    Body=bytes(json.dumps(payload), encoding="utf-8"),
    ContentType='application/json',
)

response_json = json.loads(response["Body"].read().decode())

In [ ]:
response_json

In [ ]:
%%time

payload = { # also supports analyze, verify - 
    "service": "verify",
    "img1": img1_base64, 
    "img2": img2_base64, 
}

response = sm_runtime.invoke_endpoint(
    EndpointName="face-recognition-dev-serverless",
    Body=bytes(json.dumps(payload), encoding="utf-8"),
    ContentType='application/json',
)

response_json = json.loads(response["Body"].read().decode())

In [ ]:
response_json